# Finding a Husband: Using explainable AI to define male mosquito flight differences

#### Import Libraries

In [ ]:
import sys
import time

import pandas as pd
import numpy as np

from scipy import signal, stats
from statsmodels.stats.multitest import multipletests
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn import metrics
import tqdm

np.random.seed(0)
sys.path.append('H:/Documents/PhD/mosquito-swarms/anomalies-in-swarming/src')

import reading
import preprocessing
import features

#### Loading Data

In [ ]:
males, couples = reading.load_all_files()

males = preprocessing.remove_low_length_tracks(males)
couples = preprocessing.remove_low_length_tracks(couples)


def add_velocity(dataset):
    count = 0
    for trial_index, trial in enumerate(dataset):
        for track_index, track in enumerate(trial):
            x_dot, y_dot, z_dot = preprocessing.velocity(track)
            dataset[trial_index][track_index][:, 3] = np.append(x_dot, [np.nan,np.nan])
            dataset[trial_index][track_index][:, 4] = np.append(y_dot, [np.nan,np.nan])
            dataset[trial_index][track_index][:, 5] = np.append(z_dot, [np.nan,np.nan])
            count += 1 
    return dataset

males = add_velocity(males)
couples = add_velocity(couples)


df = pd.read_excel('H:/Documents/PhD/mosquito-swarms/anomalies-in-swarming/data/search-to-pursuit/data.ods')

f = df[df['id'] == 'F']
fm = df[df['id'] == 'FM']

def format(df):
    trials = []
    for trial in df['seq'].unique():
        tracks = []
        for mossie_id in df[df['seq'] == trial]['mqid'].unique():
            tracks.append(
                df[df['seq'] == trial][df['mqid'] == mossie_id][['p1','p2','p3']].values
                )
        trials.append(tracks)
    return trials

f_df = format(f)
fm_df = format(fm)

count = 0
for trial_index, trial in enumerate(f_df):
    for track_index, track in enumerate(trial):
        f_df[trial_index][track_index] = preprocessing.append_vel(track)
        count += 1 

count = 0
for trial_index, trial in enumerate(fm_df):
    for track_index, track in enumerate(trial):
        fm_df[trial_index][track_index] = preprocessing.append_vel(track)
        count += 1 

#### Data Preprocessing

In [ ]:
'''REMOVE TRIAL ID 5'''

males.pop(5)

In [ ]:
'''Seperating tracks using a sliding window method'''

def window(track, size, over_lap):        
    start = 0
    end = size
    tracks = []
    while end < len(track):
        tracks.append(
            track[start:end]
        )
        start += (size - over_lap)
        end += (size - over_lap)
    return tracks

def segment_tracks(dataset, size, over_lap):
    split_dataset = []
    trial_id = 0
    while trial_id < len(dataset):
        track_id = 0
        trial = []
        while track_id < len(dataset[trial_id]):
            if len(dataset[trial_id][track_id]) > size:
                w = window(dataset[trial_id][track_id], size, over_lap)
                if len(w) > 1:
                    trial.append(w)
            track_id += 1
        trial_id += 1
        split_dataset.append(trial)
    return split_dataset


SIZE = 40
OVER_LAP = 20

split_males = segment_tracks(males, SIZE, OVER_LAP)
split_couples = segment_tracks(couples, SIZE, OVER_LAP)
split_females = segment_tracks(f_df, SIZE, OVER_LAP)
split_focal_males = segment_tracks(fm_df, SIZE, OVER_LAP)

#### Feature Extraction

In [ ]:
def extract(dataset):
    trial_id = 0
    while trial_id < len(dataset):
        track_group = 0
        while track_group < len(dataset[trial_id]):
            track_id = 0
            while track_id < len(dataset[trial_id][track_group]):
                # Angular Velocity 
                av_1 = features.angular_velocity(dataset[trial_id][track_group][track_id], (0,1,2), time_step=(1/25), schema='central')
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(av_1, [np.nan, np.nan]), axis=1)
                
                av_2 = features.angular_velocity(dataset[trial_id][track_group][track_id], (0,1), time_step=(1/25), schema='central')
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(av_2, [np.nan, np.nan]), axis=1)
                
                av_3 = features.angular_velocity(dataset[trial_id][track_group][track_id], (1,2), time_step=(1/25), schema='central')
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(av_3, [np.nan, np.nan]), axis=1)
                
                av_4 = features.angular_velocity(dataset[trial_id][track_group][track_id], (0,2), time_step=(1/25), schema='central')
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(av_4, [np.nan, np.nan]), axis=1)
                

                # Angular Acceleration 
                aa_1 = features.angular_acceleration(dataset[trial_id][track_group][track_id], (0,1,2), time_step=(1/25), schema='central')
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(aa_1, [np.nan, np.nan, np.nan]), axis=1)
                
                aa_2 = features.angular_acceleration(dataset[trial_id][track_group][track_id], (0,1), time_step=(1/25), schema='central')
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(aa_2, [np.nan, np.nan, np.nan]), axis=1)
                
                aa_3 = features.angular_acceleration(dataset[trial_id][track_group][track_id], (1,2), time_step=(1/25), schema='central')
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(aa_3, [np.nan, np.nan, np.nan]), axis=1)

                aa_4 = features.angular_acceleration(dataset[trial_id][track_group][track_id], (0,2), time_step=(1/25), schema='central')
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(aa_4, [np.nan, np.nan, np.nan]), axis=1)

                # Direction of Flight Change 
                dof_1 = features.direction_of_flight_change(dataset[trial_id][track_group][track_id], (0,1,2))
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(dof_1, [np.nan,np.nan]), axis=1)

                # Centroid Distance Function
                centroid_distance_function = features.centroid_distance_function(dataset[trial_id][track_group][track_id], (0,1,2))
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), centroid_distance_function, axis=1)

                # Orthogonal Components 
                pv, tv, iv = features.orthogonal_components(dataset[trial_id][track_group][track_id], (0,1,2), time_step=(1/25), schema='central')
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(pv, [np.nan, np.nan, np.nan]), axis=1)
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(tv, [np.nan, np.nan, np.nan]), axis=1)
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(iv, [np.nan, np.nan, np.nan]), axis=1)
                
                # Absolute radial velocity
                radial_velocity = np.abs(features.velocity(dataset[trial_id][track_group][track_id], (0,1,2), time_step=(1/25), schema='central'))
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(radial_velocity, [np.nan, np.nan]), axis=1)

                # Absolute radial acceleration
                radial_acceleration = features.acceleration(dataset[trial_id][track_group][track_id], (0,1,2), time_step=(1/25), schema='central')
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(radial_acceleration, [np.nan, np.nan, np.nan]), axis=1)

                # Absolute radial jerk
                jerk = features.jerk(dataset[trial_id][track_group][track_id], (0,1,2), time_step=(1/25), schema='central')
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(jerk, [np.nan, np.nan, np.nan, np.nan]), axis=1)

                # X axial acceleration
                x = features.axial_acceleration(dataset[trial_id][track_group][track_id], 0, time_step=(1/25), schema='central')
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(x, [np.nan, np.nan, np.nan]), axis=1)

                # Y axial acceleration
                y = features.axial_acceleration(dataset[trial_id][track_group][track_id], 1, time_step=(1/25), schema='central')
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(y, [np.nan, np.nan, np.nan]), axis=1)

                # Z axial acceleration
                z = features.axial_acceleration(dataset[trial_id][track_group][track_id], 2, time_step=(1/25), schema='central')
                dataset[trial_id][track_group][track_id] = np.insert(dataset[trial_id][track_group][track_id], len(dataset[trial_id][track_group][track_id][0]), np.append(z, [np.nan, np.nan, np.nan]), axis=1)

                track_id += 1
            track_group += 1
        trial_id += 1

    return dataset

males = extract(split_males)
couples = extract(split_couples)
f_df = extract(split_females)
fm_df = extract(split_focal_males)


In [ ]:
'''Generate statistics of each metric from each position'''

def track_stats(track, indexes, columns):
    f_stats = dict()
    for index, col in enumerate(columns):
        elements = track[:, indexes[index]]
        elements = elements[~np.isinf(elements)]
        elements = elements[~np.isnan(elements)]
        f_stats[col + ' (mean)'] = np.mean(elements)
        f_stats[col + ' (median)'] = np.median(elements)
        f_stats[col + ' (standard deviation)'] = np.std(elements)
        f_stats[col + ' (kurtosis)'] = stats.kurtosis(elements)
        f_stats[col + ' (skewness)'] = stats.skew(elements)
        f_stats[col + ' (no of local minima)'] = signal.argrelextrema(elements, np.less)[0].shape[0]
        f_stats[col + ' (no of local maxima)'] = signal.argrelextrema(elements, np.greater)[0].shape[0]
        f_stats[col + ' (no of zero-crossings)'] = len(np.where(np.diff(np.sign(elements)))[0])
        try:
            f_stats[col + ' (1st quartile)'] = np.percentile(elements, 25)
            f_stats[col + ' (3rd quartile)'] = np.percentile(elements, 75)
        except:
            f_stats[col + ' (1st quartile)'] = np.nan
            f_stats[col + ' (3rd quartile)'] = np.nan

    return f_stats


def remove_nans(df):
    columns_to_drop = df.columns.to_series()[np.isinf(df).any()]
    for column in columns_to_drop:
        df = df.drop(columns=str(column))

    columns_to_drop = df.columns.to_series()[np.isnan(df).any()]
    for column in columns_to_drop:
        df = df.drop(columns=str(column))
    
    indexes = df[df.isna().any(axis=1)].index
    df = df.drop(index=indexes)
    return df


def compute_segment_statistics(dictionary, dataset, indexes, feature_columns):
    for trial in dataset:
        for group in trial:
            for track in group:
                data = track_stats(track, indexes=indexes, columns=feature_columns)
                for d in data:
                    dictionary[d].append(data[d])
    return dictionary


def add_other_features(data, pos):
    def stats(features):
        return np.mean(features), np.std(features) 

    features_dict = {
        'Straightness': [],
        'Convex Hull (volume)': [],
        'Convex Hull (surface area)': [],
        'Curvature Scale Space (mean)': [],
        'Curvature Scale Space (standard deviation)': [],
        'Fractal Dimension': [],
        'Curvature X-Y (mean)': [],
        'Curvature X-Y (standard deviation)': [],
        'Curvature Y-Z (mean)': [],
        'Curvature Y-Z (standard deviation)': [],
        'Curvature X-Z (mean)': [],
        'Curvature X-Z (standard deviation)': [],
    }

    trial_id = 0
    while trial_id < len(data):
        track_group = 0
        while track_group < len(data[trial_id]):
            track_id = 0
            while track_id < len(data[trial_id][track_group]):
                features_dict['Straightness'].append(features.straightness(data[trial_id][track_group][track_id], pos))
                features_dict['Convex Hull (volume)'].append(features.convex_hull_area(data[trial_id][track_group][track_id], pos))
                features_dict['Convex Hull (surface area)'].append(features.convex_hull_perimeter(data[trial_id][track_group][track_id], pos))
                features_dict['Fractal Dimension'].append(features.fractal_dimension(data[trial_id][track_group][track_id], pos))
                css_mean, css_std = stats(features.curvature_scale_space(data[trial_id][track_group][track_id], pos))
                features_dict['Curvature Scale Space (mean)'].append(css_mean)
                features_dict['Curvature Scale Space (standard deviation)'].append(css_std)
                c1_mean, c1_std = stats(features.curvature(data[trial_id][track_group][track_id], (0,1), time_step=(1/25)))
                c2_mean, c2_std = stats(features.curvature(data[trial_id][track_group][track_id], (1,2), time_step=(1/25)))
                c3_mean, c3_std = stats(features.curvature(data[trial_id][track_group][track_id], (0,2), time_step=(1/25)))
                features_dict['Curvature X-Y (mean)'].append(c1_mean)
                features_dict['Curvature X-Y (standard deviation)'].append(c1_std)
                features_dict['Curvature Y-Z (mean)'].append(c2_mean)
                features_dict['Curvature Y-Z (standard deviation)'].append(c2_std)
                features_dict['Curvature X-Z (mean)'].append(c3_mean)
                features_dict['Curvature X-Z (standard deviation)'].append(c3_std)
                track_id += 1
            track_group += 1
        trial_id += 1
    
    feat_df = pd.DataFrame(data=features_dict)
    return feat_df


feature_columns = [
    'X Velocity',
    'Y Velocity',
    'Z Velocity',
    'Angular Velocity',
    'Angular Velocity X-Y',
    'Angular Velocity Y-Z',
    'Angular Velocity X-Z',
    'Angular Acceleration',
    'Angular Acceleration X-Y',
    'Angular Acceleration Y-Z',
    'Angular Acceleration X-Z',
    'Angle of Flight', 
    'Centroid Distance Function',
    'Persistence Velocity',
    'Turning Velocity',
    'Inclination Velocity',
    'Radial Velocity',
    'Radial Acceleration',
    'Radial Jerk',
    'X Acceleration',
    'Y Acceleration',
    'Z Acceleration'
]   

indexes = [i for i in range(3, len(feature_columns)+3)]  
feature_stats = ['mean','median','standard deviation', 'kurtosis', 'skewness','no of local minima','no of local maxima','no of zero-crossings', '1st quartile', '3rd quartile'] 


In [ ]:
male_track_statistics = dict()
couple_track_statistics = dict()
female_track_statistics = dict()
focal_male_track_statistics = dict()

for col in feature_columns:
    for stat in feature_stats:
        male_track_statistics[f'{col} ({stat})'] = []
        couple_track_statistics[f'{col} ({stat})'] = []
        female_track_statistics[f'{col} ({stat})'] = []
        focal_male_track_statistics[f'{col} ({stat})'] = []


male_track_statistics = compute_segment_statistics(male_track_statistics, males, indexes, feature_columns)
couple_track_statistics = compute_segment_statistics(couple_track_statistics, couples, indexes, feature_columns)
female_track_statistics = compute_segment_statistics(female_track_statistics, f_df, indexes, feature_columns)
focal_male_track_statistics = compute_segment_statistics(focal_male_track_statistics, fm_df, indexes, feature_columns)

df_males = pd.DataFrame(data=male_track_statistics)
df_couples = pd.DataFrame(data=couple_track_statistics)
df_f = pd.DataFrame(data=female_track_statistics)
df_fm = pd.DataFrame(data=focal_male_track_statistics)

In [ ]:
'''Add other track features'''

df_males = pd.concat([df_males, add_other_features(males, (0,1,2))], axis=1)
df_couples = pd.concat([df_couples, add_other_features(couples, (0,1,2))], axis=1)
df_f = pd.concat([df_f, add_other_features(f_df, (0,1,2))], axis=1)
df_fm = pd.concat([df_fm, add_other_features(fm_df, (0,1,2))], axis=1)

In [ ]:
df_males = remove_nans(df_males)
df_couples = remove_nans(df_couples)
df_f = remove_nans(df_f)
df_fm = remove_nans(df_fm)

In [ ]:
'''Remove columns not in both'''

similar = list(set(df_couples.columns.values) & set(df_males.columns.values) & set(df_f.columns.values) & set(df_fm.columns.values))
df_couples = df_couples[similar]
df_males = df_males[similar]
df_f = df_f[similar]
df_fm = df_fm[similar]

In [ ]:
'''Adding trials to Couple data for FS - test split'''

def add_track_and_trial_id(df, dataset):
    seq = []
    track_group = []
    count = 0
    for trial in range(len(dataset)):
        for group in range(len(dataset[trial])):
            for i in range(len(dataset[trial][group])):
                seq.append(trial)
                track_group.append(count)
            count += 1
    df['seq'] = seq
    df['track_group'] = track_group
    return df

df_males = add_track_and_trial_id(df_males, split_males)
df_couples = add_track_and_trial_id(df_couples, split_couples)
df_f = add_track_and_trial_id(df_f, split_females)
df_fm = add_track_and_trial_id(df_fm, split_focal_males)

#### Feature Selection

In [ ]:
len(df_males.columns)

In [ ]:
print(len(df_males), len(df_couples), len(df_f), len(df_fm))#

In [ ]:
print(len(df_males['track_group'].unique()), len(df_couples['track_group'].unique()), len(df_f['track_group'].unique()), len(df_fm['track_group'].unique()))#

In [ ]:
'''Split Couple data'''
df_males_fs = df_males[(df_males['seq'] == 4) | (df_males['seq'] == 9) | (df_males['seq'] == 3)]
df_males_test = df_males[~((df_males['seq'] == 4) | (df_males['seq'] == 9) | (df_males['seq'] == 3))]
df_couple_fs = df_couples[(df_couples['seq'] == 2) | (df_couples['seq'] == 6) ].drop(columns=['seq'])
df_couple_test = df_couples[~((df_couples['seq'] == 2) | (df_couples['seq'] == 6))].drop(columns=['seq'])

In [ ]:
print(len(df_males_fs), len(df_couple_fs))

In [ ]:
'''Conducting U-Test to determine whether there is any statistical difference between couples and males trajectory'''

x = df_males_fs.drop(columns=['seq', 'track_group'], errors='ignore')
y = df_couple_fs.drop(columns=['track_group'], errors='ignore')

results = stats.mannwhitneyu(x, y)
rej, pvals, a1, a2 = multipletests(results[1], alpha=0.01, method='holm')

cols = {"columns": [], "p_values": [], 'raw_p':[]}

for index, column in enumerate(x.columns.values):
    if rej[index] == True:
        cols['columns'].append(column)
        cols['p_values'].append(pvals[index])
        cols['raw_p'].append(results[1][index])

df = pd.DataFrame(data=cols)
df


In [ ]:
'''Removing Correlated features'''
df_hyp = pd.concat([df_males_fs, df_couple_fs])

df_hyp_sorted = df_hyp.reindex(sorted(df_hyp.columns), axis=1)
corr_matrix = df_hyp_sorted.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]

In [ ]:
'''Adding trial id to male df so that splitting by trial can occur'''

seq = []
for trial in range(len(split_males)):
    for i in range(len(split_males[trial])):
        for j in range(len(split_males[trial][i])):
            seq.append(trial)

df_males['seq'] = seq
df_males_test = df_males.drop(index=df_males_fs.index.values)
df_males_fs = df_males.drop(index=df_males_test.index.values)

In [ ]:
columns = df_couple_test[cols["columns"]].drop(columns=to_drop, errors='ignore').columns.values

In [ ]:
len(columns)

In [ ]:
for i in sorted(columns):
    print(i)

#### Storing Data

In [ ]:
'''Store Data'''

# Store all track information (males, couples, focal-males, females)
df_males_test.to_pickle('data/df_males_test.pkl')
df_males_fs.to_pickle('data/df_males_fs.pkl')
df_couple_test.to_pickle('data/df_couples_test.pkl')
df_couple_fs.to_pickle('data/df_couples_fs.pkl')
df_f.to_pickle('data/df_females.pkl')
df_fm.to_pickle('data/df_focal_males.pkl')


np.save('data/males.npy', males)
np.save('data/couple.npy', couples)
np.save('data/f_df.npy', f_df)
np.save('data/fm_df.npy', fm_df)

# Store p-values
np.save('data/non_adj_pvals.npy', results[1])
np.save('data/all_pvals.npy', pvals)
df.to_pickle('data/selected_pvals.pkl')

# Store selected columns
columns = df_couple_test[cols["columns"]].drop(columns=to_drop, errors='ignore').columns.values
np.save('data/columns.npy', columns)

### Loading Data

In [ ]:
'''Load Data'''

df_males_test = pd.read_pickle('data/df_males_test.pkl')
df_males_fs = pd.read_pickle('data/df_males_fs.pkl')
df_couple_test = pd.read_pickle('data/df_couples_test.pkl')
df_couple_fs = pd.read_pickle('data/df_couples_fs.pkl')
df_f = pd.read_pickle('data/df_females.pkl')
df_fm = pd.read_pickle('data/df_focal_males.pkl')

males = np.load('data/males.npy', allow_pickle=True)
couples = np.load('data/couple.npy', allow_pickle=True)
f_df = np.load('data/f_df.npy', allow_pickle=True)
fm_df = np.load('data/fm_df.npy', allow_pickle=True)

df_males = pd.concat([df_males_fs, df_males_test])
df_couples = pd.concat([df_couple_test, df_couple_fs])

columns = np.load('data/columns.npy', allow_pickle=True)
pvals = pd.read_pickle('data/selected_pvals.pkl')
non_adj_pvals = np.load('data/non_adj_pvals.npy', allow_pickle=True)
all_pvals = np.load('data/all_pvals.npy', allow_pickle=True)

#### Modelling

In [ ]:
def compute_logistic_array(arr):
    return [1/(1 + np.exp(-x)) for x in arr]

def get_mode(_set, predicts, decisions):
    predictions = []
    avg_decision = []
    unique_track_groups = _set.unique()
    for val in unique_track_groups:
        indexes = np.where(_set == val)
        preds = predicts[indexes]
        avg_decision.append(np.mean(decisions[indexes]))
        preds = np.array(np.sign(np.mean(decisions[indexes])))
        preds[preds == 0] = 1
        predictions.append(preds)
    return predictions, avg_decision

In [ ]:
'''
CROSS VALIDATION 
'''
all_trials = df_males_test['seq'].unique()

male_train_predictions = {'predictions': [], 'accuracy': []}
male_test_predictions = {'predictions': [], 'accuracy': []}
couple_predictions = {'predictions': [],  'accuracy': []}
female_predictions = {'predictions': [], 'accuracy': []}
focal_male_predictions = {'predictions': [], 'accuracy': []}
all_test_predictions = {
    'true': [], 'preds': [],
    'accuracy': [], 'roc auc':[], 
    'f1 score (male)':[], 'recall (male)':[], 'precision (male)':[], 
    'f1 score (non-male)':[], 'recall (non-male)':[], 'precision (non-male)':[],
    'f1 score (avg)':[], 'recall (avg)':[], 'precision (avg)':[], }
roc_curve = {'fpr': [], 'tpr': []}
pr_curve = {'m_precision': [], 'm_recall': [], 'm_auc':[], 'm_pn':[], 'n_precision': [], 'n_recall': [], 'n_auc':[], 'n_pn':[], 'auc':[]}

shap_train = []
shap_test = []

seq_test = []
seq_train = []

nu = 0.2
kernel = 'rbf'
gamma = (1/len(columns))**2
degree = 3

a1 = all_trials
a2 = all_trials
for val1 in tqdm.tqdm(a1):
    a2 = a2[a2 != val1]
    for val2 in a2:
        if val1 != val2:
            #print(f' ----- {val1} - {val2} ----')
            train = df_males_test[~((df_males_test['seq'] == val1) | (df_males_test['seq'] == val2))]
            test = df_males_test[((df_males_test['seq'] == val1) | (df_males_test['seq'] == val2))]

            seq_test.append([val1, val2])
            seq_train.append([x for x in all_trials if x not in [val1, val2]])
    
            scaler = StandardScaler()
            train_standard = scaler.fit_transform(train[columns])
            train_standard = pd.DataFrame(train_standard, index=train.index, columns=columns)

            test_standard = scaler.transform(test[columns])
            test_standard = pd.DataFrame(test_standard, index=test.index, columns=columns)

            df_couple_test_standard = scaler.transform(df_couple_test[columns])
            df_couple_test_standard = pd.DataFrame(df_couple_test_standard, index=df_couple_test.index, columns=columns)

            df_f_scaler = scaler.transform(df_f[columns])
            df_f_scaler = pd.DataFrame(df_f_scaler, index=df_f.index, columns=columns)

            focal_male_standard = scaler.transform(df_fm[columns])
            focal_male_standard = pd.DataFrame(focal_male_standard, index=df_fm.index, columns=columns)

            clf = OneClassSVM(
                nu=nu,
                kernel=kernel,
                gamma=gamma,
                degree=degree
            ).fit(train_standard)

            # ---------- TRAIN SET ---------- 
            male_train_predictions['predictions'].append(clf.predict(train_standard))
            preds, scores = get_mode(train['track_group'], clf.predict(train_standard), clf.decision_function(train_standard))
            male_train_predictions['accuracy'].append(metrics.accuracy_score([1 for l in range(len(preds))], preds))

            # ---------- TEST SET ---------- 
            male_test_predictions['predictions'].append(clf.predict(test_standard))
            preds, scores = get_mode(test['track_group'], clf.predict(test_standard), clf.decision_function(test_standard))
            male_test_predictions['accuracy'].append(metrics.accuracy_score([1 for l in range(len(preds))], preds))

            # ---------- COUPLE SET ---------- 
            couple_predictions['predictions'].append(clf.predict(df_couple_test_standard))
            preds, scores = get_mode(df_couple_test['track_group'], clf.predict(df_couple_test_standard), clf.decision_function(df_couple_test_standard))
            couple_predictions['accuracy'].append(metrics.accuracy_score([-1 for l in range(len(preds))], preds))

            # ---------- FEMALE SET ---------- 
            female_predictions['predictions'].append(clf.predict(df_f_scaler))
            preds, scores = get_mode(df_f['track_group'], clf.predict(df_f_scaler), clf.decision_function(df_f_scaler))
            female_predictions['accuracy'].append(metrics.accuracy_score([-1 for l in range(len(preds))], preds))

            # ---------- FOCAL MALE SET ---------- 
            focal_male_predictions['predictions'].append(clf.predict(focal_male_standard))
            preds, scores = get_mode(df_fm['track_group'], clf.predict(focal_male_standard), clf.decision_function(focal_male_standard))
            focal_male_predictions['accuracy'].append(metrics.accuracy_score([1 for l in range(len(preds))], preds))

            # ------------ ALL TEST SET ------------
            full_test_set = pd.concat([test[columns], df_couple_test[columns], df_f[columns], df_fm[columns]])
            full_test_set_scaled = scaler.transform(full_test_set)
            full_test_set = pd.DataFrame(full_test_set_scaled, index=full_test_set.index, columns=full_test_set.columns)
            full_test_groups = pd.concat([
                test['track_group'].apply(lambda row: f'm{row}'),
                df_couple_test['track_group'].apply(lambda row: f'c{row}'),
                df_f['track_group'].apply(lambda row: f'f{row}'),
                df_fm['track_group'].apply(lambda row: f'fm{row}')])

            all_track_targets = [1 for _ in range(len(test['track_group'].unique()))] + [-1 for _ in range(len(df_couple_test['track_group'].unique()))] + [-1 for _ in range(len(df_f['track_group'].unique()))] + [1 for _ in range(len(df_fm['track_group'].unique()))]

            preds, scores = get_mode(
                full_test_groups,
                clf.predict(full_test_set), 
                clf.decision_function(full_test_set), 
            )
            all_test_predictions['true'].append(all_track_targets)
            all_test_predictions['preds'].append(preds)

            all_test_predictions['accuracy'].append(metrics.balanced_accuracy_score(all_track_targets, preds))
            all_test_predictions['roc auc'].append(metrics.roc_auc_score(np.array(all_track_targets), compute_logistic_array(np.array(scores))))

            all_test_predictions['f1 score (male)'].append(metrics.f1_score(all_track_targets, preds))
            all_test_predictions['recall (male)'].append(metrics.recall_score(all_track_targets, preds))
            all_test_predictions['precision (male)'].append(metrics.precision_score(all_track_targets, preds))

            all_test_predictions['f1 score (non-male)'].append(metrics.f1_score(all_track_targets, preds, pos_label=-1))
            all_test_predictions['recall (non-male)'].append(metrics.recall_score(all_track_targets, preds, pos_label=-1))
            all_test_predictions['precision (non-male)'].append(metrics.precision_score(all_track_targets, preds, pos_label=-1))
            
            all_test_predictions['f1 score (avg)'].append((all_test_predictions['f1 score (male)'][-1] + all_test_predictions['f1 score (non-male)'][-1])/2)
            all_test_predictions['recall (avg)'].append((all_test_predictions['recall (male)'][-1] + all_test_predictions['recall (non-male)'][-1])/2)
            all_test_predictions['precision (avg)'].append((all_test_predictions['precision (male)'][-1] + all_test_predictions['precision (non-male)'][-1])/2)

            fpr, tpr, _ = metrics.roc_curve(np.array(all_track_targets), compute_logistic_array(scores))
            roc_curve['fpr'].append(fpr)
            roc_curve['tpr'].append(tpr)

            shap_train.append(train_standard)
            shap_test.append(full_test_set)

            precision, recall, thresholds = metrics.precision_recall_curve(all_track_targets, compute_logistic_array(scores))
            pr_curve['m_precision'].append(precision)
            pr_curve['m_recall'].append(recall)
            pr_curve['m_auc'].append(metrics.auc(recall, precision))
            pr_curve['m_pn'].append(len([i for i in all_track_targets if i == 1])/len(all_track_targets))

            precision, recall, thresholds = metrics.precision_recall_curve(np.array(all_track_targets)*-1, [1 - ar for ar in compute_logistic_array(np.array(scores))])
            pr_curve['n_precision'].append(precision)
            pr_curve['n_recall'].append(recall)
            pr_curve['n_auc'].append(metrics.auc(recall, precision))
            pr_curve['n_pn'].append(len([i for i in np.array(all_track_targets) if i == -1])/len(all_track_targets))

            pr_curve['auc'].append((pr_curve['m_auc'][-1] + pr_curve['n_auc'][-1])/2)


In [ ]:
np.save('shap_train.npy', np.array(shap_train))
np.save('shap_test.npy', np.array(shap_test))

In [ ]:
print(' -- TRAIN SET (MALES) --')
print(f'Accuracy: {round(sum(male_train_predictions["accuracy"])/len(male_train_predictions["accuracy"]), 3)} ({round(np.percentile(male_train_predictions["accuracy"], 2.5), 3)} - {round(np.percentile(male_train_predictions["accuracy"], 97.5), 3)})')
print('\n -- TEST SET (MALES) --')
print(f'Accuracy: {round(sum(male_test_predictions["accuracy"])/len(male_test_predictions["accuracy"]), 3)} ({round(np.percentile(male_test_predictions["accuracy"], 2.5), 3)} - {round(np.percentile(male_test_predictions["accuracy"], 97.5), 3)})')
print('\n -- COUPLES --')
print(f'Accuracy: {round(sum(couple_predictions["accuracy"])/len(couple_predictions["accuracy"]), 3)} ({round(np.percentile(couple_predictions["accuracy"], 2.5), 3)} - {round(np.percentile(couple_predictions["accuracy"], 97.5), 3)})')
print('\n -- FEMALES -- ')
print(f'Accuracy: {round(sum(female_predictions["accuracy"])/len(female_predictions["accuracy"]), 3)} ({round(np.percentile(female_predictions["accuracy"], 2.5), 3)} - {round(np.percentile(female_predictions["accuracy"], 97.5), 3)})')
print('\n -- FOCAL MALES -- ')
print(f'Accuracy: {round(sum(focal_male_predictions["accuracy"])/len(focal_male_predictions["accuracy"]), 3)} ({round(np.percentile(focal_male_predictions["accuracy"], 2.5), 3)} - {round(np.percentile(focal_male_predictions["accuracy"], 97.5), 3)})')

print('\n -- TOTAL DATASET --')
print(f'BALANCED ACCURACY: {round(sum(all_test_predictions["accuracy"])/len(all_test_predictions["accuracy"]), 3)} ({round(np.percentile(all_test_predictions["accuracy"], 2.5), 3)} - {round(np.percentile(all_test_predictions["accuracy"], 97.5), 3)})')
print(f'ROC AUC: {round(sum(all_test_predictions["roc auc"])/len(all_test_predictions["roc auc"]), 3)} ({round(np.percentile(all_test_predictions["roc auc"], 2.5), 3)} - {round(np.percentile(all_test_predictions["roc auc"], 97.5), 3)})')

print(f'\nF1 SCORE (avg): {round(sum(all_test_predictions["f1 score (avg)"])/len(all_test_predictions["f1 score (avg)"]), 3)} ({round(np.percentile(all_test_predictions["f1 score (avg)"], 2.5), 3)} - {round(np.percentile(all_test_predictions["f1 score (avg)"], 97.5), 3)})')
print(f'F1 SCORE (male): {round(sum(all_test_predictions["f1 score (male)"])/len(all_test_predictions["f1 score (male)"]), 3)} ({round(np.percentile(all_test_predictions["f1 score (male)"], 2.5), 3)} - {round(np.percentile(all_test_predictions["f1 score (male)"], 97.5), 3)})')
print(f'F1 SCORE (non-male): {round(sum(all_test_predictions["f1 score (non-male)"])/len(all_test_predictions["f1 score (non-male)"]), 3)} ({round(np.percentile(all_test_predictions["f1 score (non-male)"], 2.5), 3)} - {round(np.percentile(all_test_predictions["f1 score (non-male)"], 97.5), 3)})')

print(f'\nRECALL (avg): {round(sum(all_test_predictions["recall (avg)"])/len(all_test_predictions["recall (avg)"]), 3)} ({round(np.percentile(all_test_predictions["recall (avg)"], 2.5), 3)} - {round(np.percentile(all_test_predictions["recall (avg)"], 97.5), 3)})')
print(f'RECALL (male): {round(sum(all_test_predictions["recall (male)"])/len(all_test_predictions["recall (male)"]), 3)} ({round(np.percentile(all_test_predictions["recall (male)"], 2.5), 3)} - {round(np.percentile(all_test_predictions["recall (male)"], 97.5), 3)})')
print(f'RECALL (non-male): {round(sum(all_test_predictions["recall (non-male)"])/len(all_test_predictions["recall (non-male)"]), 3)} ({round(np.percentile(all_test_predictions["recall (non-male)"], 2.5), 3)} - {round(np.percentile(all_test_predictions["recall (non-male)"], 97.5), 3)})')

print(f'\nPRECISION (avg): {round(sum(all_test_predictions["precision (avg)"])/len(all_test_predictions["precision (avg)"]), 3)} ({round(np.percentile(all_test_predictions["precision (avg)"], 2.5), 3)} - {round(np.percentile(all_test_predictions["precision (avg)"], 97.5), 3)})')
print(f'PRECISION (male): {round(sum(all_test_predictions["precision (male)"])/len(all_test_predictions["precision (male)"]), 3)} ({round(np.percentile(all_test_predictions["precision (male)"], 2.5), 3)} - {round(np.percentile(all_test_predictions["precision (male)"], 97.5), 3)})')
print(f'PRECISION (non-male): {round(sum(all_test_predictions["precision (non-male)"])/len(all_test_predictions["precision (non-male)"]), 3)} ({round(np.percentile(all_test_predictions["precision (non-male)"], 2.5), 3)} - {round(np.percentile(all_test_predictions["precision (non-male)"], 97.5), 3)})')

print(f'\nPR AUC (avg): {round(sum(pr_curve["auc"])/len(pr_curve["auc"]), 3)} ({round(np.percentile(pr_curve["auc"], 2.5), 3)} - {round(np.percentile(pr_curve["auc"], 97.5), 3)})')
print(f'PR AUC (male): {round(sum(pr_curve["m_auc"])/len(pr_curve["m_auc"]), 3)} ({round(np.percentile(pr_curve["m_auc"], 2.5), 3)} - {round(np.percentile(pr_curve["m_auc"], 97.5), 3)})')
print(f'PR AUC (non-male): {round(sum(pr_curve["n_auc"])/len(pr_curve["n_auc"]), 3)} ({round(np.percentile(pr_curve["n_auc"], 2.5), 3)} - {round(np.percentile(pr_curve["n_auc"], 97.5), 3)})')


#### Hyperparameter Tuning

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


In [ ]:
'''
HYPERPARAMETER TUNING
'''

data = {'nu':[], 'kernel':[], 'gamma':[], 'degree':[], 'accuracy': [], 'roc auc':[]}
all_trials = df_males_fs['seq'].unique()

for nu in tqdm.tqdm([i/100 for i in range(1, 40, 1)]):
    for kernel in ['rbf']:
        for degree in [3]:
            for gamma in ['scale', 'auto', 1/(len(columns)**2)]:
                a1 = all_trials
                a2 = all_trials
                accuracy_vals = []
                roc_auc_vals = []

                for val1 in a1:
                    a2 = a2[a2 != val1]
                    for val2 in a2:
                        if val1 != val2:
                            train = df_males_fs[((df_males_fs['seq'] == val1) | (df_males_fs['seq'] == val2))]
                            test = df_males_fs[~((df_males_fs['seq'] == val1) | (df_males_fs['seq'] == val2))]

                            scaler = StandardScaler()
                            train_standard = scaler.fit_transform(train[columns])
                            train_standard = pd.DataFrame(train_standard, columns=columns, index=train.index)

                            clf = OneClassSVM(
                                nu=nu,
                                kernel=kernel,
                                degree=degree, 
                                gamma=gamma
                            ).fit(train_standard)

                            # ------------ ALL TEST SET ------------
                            full_test = pd.concat([test[columns], df_couple_fs[columns]])
                            full_test_set = scaler.transform(full_test)
                            full_test_set = pd.DataFrame(full_test_set, index=full_test.index, columns=columns)
                            full_test_groups = pd.concat([
                                test['track_group'].apply(lambda row: f'm{row}'),
                                df_couple_fs['track_group'].apply(lambda row: f'c{row}')])

                            all_track_targets = [1 for _ in range(len(test['track_group'].unique()))] + [-1 for _ in range(len(df_couple_fs['track_group'].unique()))] 

                            preds, scores = get_mode(
                                full_test_groups,
                                clf.predict(full_test_set), 
                                clf.decision_function(full_test_set), 
                            )
                            accuracy_vals.append(metrics.balanced_accuracy_score(all_track_targets, preds))
                            roc_auc_vals.append(metrics.roc_auc_score(np.array(all_track_targets), compute_logistic_array(np.array(scores))))

                data['nu'].append(nu)
                data['kernel'].append(kernel)
                data['degree'].append(degree)
                data['gamma'].append(gamma)
                data['accuracy'].append(np.mean(accuracy_vals))
                data['roc auc'].append(np.mean(roc_auc_vals))


In [ ]:
hyp_df = pd.DataFrame(data)

In [ ]:
hyp_df.to_pickle('hyp_df.pkl')

In [ ]:
hyp_df = pd.read_pickle('results/tuned and standardised/hyp_df.pkl')

In [ ]:
best_acc = hyp_df.sort_values(by='accuracy').iloc[-1]['accuracy']
hyp_df[(hyp_df['accuracy'] == best_acc)]

In [ ]:
hyp_df.sort_values(by='accuracy')[-20:]

#### Analysis

In [ ]:
'''EXCEL FILE OF ALL FOLD SCORES'''

import openpyxl

wb = openpyxl.Workbook()
sheet = wb.create_sheet()

cols = ['fold', 'test trials', 'train trials', 'accuracy (male train)', 'accuracy (male test)',
    'accuracy (couple test)', 'balanced accuracy (all)', 'roc auc (all)']

for i, col in enumerate(cols):
    sheet.cell(row=1, column=i+1).value = col


for fold in range(len(all_test_predictions['accuracy'])):
    sheet.cell(row=fold+2, column=1).value = fold
    sheet.cell(row=fold+2, column=2).value = str(seq_test[fold])
    sheet.cell(row=fold+2, column=3).value = str(seq_train[fold])
    sheet.cell(row=fold+2, column=4).value = male_train_predictions['accuracy'][fold]
    sheet.cell(row=fold+2, column=5).value = male_test_predictions['accuracy'][fold]
    sheet.cell(row=fold+2, column=6).value = couple_predictions['accuracy'][fold]
    sheet.cell(row=fold+2, column=7).value = all_test_predictions['accuracy'][fold]
    sheet.cell(row=fold+2, column=8).value = all_test_predictions['roc auc'][fold]

wb.save("all-folds.xlsx")


In [ ]:
from sklearn.metrics import confusion_matrix as con_mat
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import chain

y_test =  list(chain.from_iterable(all_test_predictions['true']))
y_pred =  list(chain.from_iterable(all_test_predictions['preds']))

plt.rcParams.update({'font.size': 22})
plt.figure(figsize=(8, 6), dpi=400)
con = con_mat(y_test, y_pred)
cmap = sns.light_palette("#dd7301", as_cmap=True)
sns.heatmap(con, annot=True, cmap=cmap, fmt=".1f")
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.xticks([0.5,1.5], ['Non-Male', 'Male'])
plt.yticks([0.5,1.5], ['Non-Male', 'Male'])
plt.show()

In [ ]:
'''ROC Curve'''

import matplotlib.pyplot as plt

plt.figure(figsize=(10,10), dpi=400)
plt.rcParams.update({'font.size': 16})
lw = 2
tprs = []
base_fpr = np.linspace(0, 1, 101)

for index in range(len(roc_curve['tpr'])):
    plt.plot(
        roc_curve['fpr'][index],
        roc_curve['tpr'][index],
        color="blue",
        alpha=0.15,
        lw=lw,
    )
    tpr = np.interp(base_fpr, roc_curve['fpr'][index], roc_curve['tpr'][index])
    tpr[0] = 0.0
    tprs.append(tpr)

tprs = np.array(tprs)
mean_tprs = tprs.mean(axis=0)
std = tprs.std(axis=0)

tprs_upper = np.minimum(mean_tprs + std, 1)
tprs_lower = mean_tprs - std


plt.plot(base_fpr, mean_tprs, 'b')
plt.fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.3)


plt.plot([0, 1], [0, 1], color="black", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.xlim((0,1))
plt.ylim((0,1))
plt.title("Receiver Operating Characteristic (ROC) Curve")
#plt.legend(loc="lower right")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10), dpi=400)
plt.rcParams.update({'font.size': 16})
lw = 2
tprs = []
base_fpr = np.linspace(0, 1, 101)

line_type = ['-', '--', '-.']
colours = ['blue', 'green']

best = 1
worst = 22

for i, index in enumerate([best, worst]):
    plt.plot(
        pr_curve['m_recall'][index],
        pr_curve['m_precision'][index],
        color=colours[i],
        lw=lw,
    )
    plt.plot(
        base_fpr,
        [pr_curve['m_pn'][index] for i in base_fpr],
        color=colours[i],
        alpha=0.4,
        linestyle='--'
    )

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.xlim((0,1))
plt.ylim((0,1))
plt.legend([f'Best fold (AUC = {round(pr_curve["m_auc"][best], 3)})', f'Best fold baseline (AUC = {round(pr_curve["m_pn"][best], 3)})', f'Worst fold (AUC = {round(pr_curve["m_auc"][worst], 3)}', f'Worst fold baseline (AUC = {round(pr_curve["m_pn"][worst], 3)})'])
plt.title(f"Precision-Recall (PR) Curve (male)")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10), dpi=400)
plt.rcParams.update({'font.size': 16})
lw = 2
tprs = []
base_fpr = np.linspace(0, 1, 101)

line_type = ['-', '--', '-.']
colours = ['blue', 'green']

best = 1
worst = 22

for i, index in enumerate([best, worst]):
    plt.plot(
        pr_curve['n_recall'][index],
        pr_curve['n_precision'][index],
        color=colours[i],
        lw=lw,
    )
    plt.plot(
        base_fpr,
        [pr_curve['n_pn'][index] for i in base_fpr],
        color=colours[i],
        alpha=0.4,
        linestyle='--'
    )

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.xlim((0,1))
plt.ylim((0,1))
plt.legend([f'Best fold (AUC = {round(pr_curve["n_auc"][best], 3)})', f'Best fold baseline (AUC = {round(pr_curve["n_pn"][best], 3)})', f'Worst fold (AUC = {round(pr_curve["n_auc"][worst], 3)}', f'Worst fold baseline (AUC = {round(pr_curve["n_pn"][worst], 3)})'])
plt.title("Precision-Recall (PR) Curve (non-male)")
plt.show()